In [80]:
import torch
from transformers import BertTokenizer, BertForSequenceClassification, AdamW
from torch.utils.data import DataLoader, TensorDataset
import pandas as pd
from sklearn.preprocessing import LabelEncoder

import pandas as pd
import numpy as np

import random

import matplotlib.pyplot as plt

import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

### LOADUP Trained Model BERT

In [81]:
dataset = pd.read_csv('./(Preprocessed)Emotion_classify_Data(Labeled).csv')

# Shuffle the dataset
dataset = dataset.sample(frac=1, random_state=42).reset_index(drop=True)

# Initialize BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Load pre-trained BERT model for sequence classification
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=dataset['Emotion'].nunique())

# Fine-tune BERT on your dataset
learning_rate = 2e-5
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)
loss_fn = torch.nn.CrossEntropyLoss()

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [82]:
emotion_labels = dataset['Emotion']

# Instantiate the label encoder
label_encoder = LabelEncoder()

# Fit the label encoder to the encoded emotion labels
label_encoder.fit(emotion_labels)

LabelEncoder()

In [83]:
# Load the checkpoint
checkpoint = torch.load('last_trained_model_checkpoint.pth', map_location=torch.device('cpu'))

# Initialize the model and optimizer
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=3)
optimizer = AdamW(model.parameters(), lr=5e-5)

# Load model and optimizer states with strict=False to ignore missing keys
model.load_state_dict(checkpoint['model_state_dict'], strict=False)
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])

# Update other necessary variables
epoch = checkpoint['epoch']
best_val_loss = checkpoint['best_val_loss']
early_stop_count = checkpoint['early_stop_count']

print(f"Model and optimizer states loaded successfully.")
print(f"Epoch: {epoch}, Best Validation Loss: {best_val_loss}, Early Stop Count: {early_stop_count}")

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

Model and optimizer states loaded successfully.
Epoch: 7, Best Validation Loss: 0.09719424509431089, Early Stop Count: 5


d:\App\Programming\python\Anaconda\lib\site-packages\transformers\optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [84]:
def predict_emotion(text, label_encoder):
    # Tokenize the text using the same tokenizer used during training
    inputs = tokenizer.encode_plus(text, add_special_tokens=True, max_length=128, padding='max_length', truncation=True, return_tensors='pt')
    input_ids = inputs['input_ids'].to(device)  # Move input to the same device as the model
    attention_mask = inputs['attention_mask'].to(device)  # Move attention mask to the same device as the model

    # Make prediction
    with torch.no_grad():
        model.eval()  # Set model to evaluation mode
        outputs = model(input_ids, attention_mask=attention_mask)
    logits = outputs.logits

    # Get predicted label
    predicted_label = torch.argmax(logits, dim=1).item()

    # Decode the predicted label using label_encoder
    predicted_emotion = label_encoder.inverse_transform([predicted_label])[0]

    return predicted_emotion, logits

# Example usage
text = "Super happy today"
predicted_emotion, x = predict_emotion(text, label_encoder)
print(f'Predicted Emotion: {predicted_emotion}')
print(f'Predicted Label: {x}')

Predicted Emotion: joy
Predicted Label: tensor([[-2.6676, -2.7545,  6.4300]])


### Read the Quote

In [85]:
quotesDF = pd.read_csv('./(Preprocessed)quotes.csv')
quotesDF

,Quote,Author,emotion
0,"Don't cry because it's over, smile because it ...",Dr. Seuss,joy
1,"I'm selfish, impatient and a little insecure. ...",Marilyn Monroe,anger
2,Be yourself; everyone else is already taken.,Oscar Wilde,anger
3,Two things are infinite: the universe and huma...,Albert Einstein,joy
4,"Be who you are and say what you feel, because ...",Bernard M. Baruch,joy
...,...,...,...
36932,"In Buddhism, they say attachment to anything o...",Jason Mraz,joy
36933,I love British humor. It's just so - surreal.,Beck,fear
36934,I've got a sense of humor. I'm a funny guy.,Daryl Hall,joy
36935,"Humor is such a wonderful thing, helping you r...",Lynda Barry,joy


In [86]:
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

def find_most_similar_quote(input_sentence, quotes_df, input_emotion, emotion_weight=1.1):
    # Combine all quotes into a list
    quotes = quotes_df['Quote'].tolist()
    authors = quotes_df['Author'].tolist()
    emotions = quotes_df['emotion'].tolist()

    # Initialize the vectorizer and fit it on all quotes
    vectorizer = TfidfVectorizer()
    tfidf_matrix = vectorizer.fit_transform(quotes)

    # Vectorize the input sentence
    input_vec = vectorizer.transform([input_sentence])

    # Calculate cosine similarity between the input sentence and all quotes
    cosine_similarities = cosine_similarity(input_vec, tfidf_matrix).flatten()

    # Apply emotion-based weighting
    emotion_weights = np.array([emotion_weight if emotion == input_emotion else 1.0 for emotion in emotions])
    weighted_similarities = cosine_similarities * emotion_weights

     # Get the indices of the top 10 quotes with the highest similarity scores
    top_10_indices = np.argsort(weighted_similarities)[-5:]

    # Randomly select one quote from the top 10
    selected_index = random.choice(top_10_indices)

    # Return the most similar quote and its author
    return quotes[selected_index], authors[selected_index], emotions[selected_index]

In [87]:
input_sentence = "What should I do with my life?"
detected_emotion, emotion_score = predict_emotion(input_sentence, label_encoder)
most_similar_quote, author, emotionQuotes = find_most_similar_quote(input_sentence, quotesDF, detected_emotion)
print('Input sentence: ', input_sentence)
print("Detected emotion:", detected_emotion)
print("Emotion score:", emotion_score)
print("Most similar quote:", most_similar_quote)
print("Author: ", author)
print("Quotes Emotion: ", emotionQuotes)

Input sentence:  What should I do with my life?
Detected emotion: anger
Emotion score: tensor([[ 1.2938,  0.3162, -1.3738]])
Most similar quote: Don't let what you cannot do interfere with what you can do.
Author:  John Wooden
Quotes Emotion:  anger
